In [23]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [24]:
import pandas as pd
import numpy as np
import ontime as on

# Models

This class implement a generic way to load models in onTime. It is compatible with Darts, Scikit-learn and PyTorch, with an aim to add other librairies and models soon. Let's see how to use it.

## Let's generate random TimeSeries

In [25]:
# Start and end dates
start_date = pd.Timestamp('2022-01-01')
end_date = pd.Timestamp('2022-12-31')

# Make a random walk
ts = on.generators.random_walk().generate(start=start_date, end=end_date)
ts = ts.astype(np.float32)

# Make another random walk
new_ts = on.generators.random_walk().generate(start=start_date, end=end_date)
new_ts = ts.astype(np.float32)

## Using a Darts model

The model can be loaded from the desired library

In [26]:
from darts.models import BlockRNNModel

Then, it can be used with this generic interface.

In [27]:
model = on.Model(BlockRNNModel,
                 input_chunk_length=12,
                 output_chunk_length=6,
                 n_rnn_layers=2,
                 n_epochs=50
                 )

Finally, the training and inference functions are the same than in Scikit Learn for instance :

In [28]:
# To train the model
model.fit(ts)

darts.models.forecasting.torch_forecasting_model INFO  Train dataset contains 348 samples.
darts.models.forecasting.torch_forecasting_model INFO  Time series values are 32-bits; casting model to float32.
INFO: GPU available: True (cuda), used: True
lightning.pytorch.utilities.rank_zero INFO  GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
lightning.pytorch.utilities.rank_zero INFO  TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
lightning.pytorch.utilities.rank_zero INFO  HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricC

True


Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
lightning.pytorch.utilities.rank_zero INFO  `Trainer.fit` stopped: `max_epochs=50` reached.


In [29]:
# To make a prediction from train series
darts_pred = model.predict(10)

INFO: GPU available: True (cuda), used: True
lightning.pytorch.utilities.rank_zero INFO  GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
lightning.pytorch.utilities.rank_zero INFO  TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
lightning.pytorch.utilities.rank_zero INFO  HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Once trained, the model can also infer on a new series, or even on a list of series (batched inference)

In [30]:
# on a new single series
model.predict(n=10, ts=new_ts)
# on many series
model.predict(n=10, ts=[new_ts]*3)

INFO: GPU available: True (cuda), used: True
lightning.pytorch.utilities.rank_zero INFO  GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
lightning.pytorch.utilities.rank_zero INFO  TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
lightning.pytorch.utilities.rank_zero INFO  HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: GPU available: True (cuda), used: True
lightning.pytorch.utilities.rank_zero INFO  GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
lightning.pytorch.utilities.rank_zero INFO  TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
lightning.pytorch.utilities.rank_zero INFO  HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[<TimeSeries (DataArray) (time: 10, component: 1, sample: 1)> Size: 40B
 array([[[-2.114943 ]],
 
        [[-1.5462902]],
 
        [[-2.2913358]],
 
        [[-2.0579665]],
 
        [[-2.686432 ]],
 
        [[-2.4956915]],
 
        [[-2.6606524]],
 
        [[-2.825928 ]],
 
        [[-2.5841327]],
 
        [[-2.8603108]]], dtype=float32)
 Coordinates:
   * time       (time) datetime64[ns] 80B 2023-01-01 2023-01-02 ... 2023-01-10
   * component  (component) object 8B 'random_walk'
 Dimensions without coordinates: sample
 Attributes:
     static_covariates:  None
     hierarchy:          None,
 <TimeSeries (DataArray) (time: 10, component: 1, sample: 1)> Size: 40B
 array([[[-2.114943 ]],
 
        [[-1.5462902]],
 
        [[-2.2913358]],
 
        [[-2.0579665]],
 
        [[-2.686432 ]],
 
        [[-2.4956915]],
 
        [[-2.6606524]],
 
        [[-2.825928 ]],
 
        [[-2.5841327]],
 
        [[-2.8603108]]], dtype=float32)
 Coordinates:
   * time       (time) datetime64[n

In [31]:
darts_pred = darts_pred.rename({'random_walk': 'darts prediction'}) # for plotting

## Using a Scikit-learn API compatible model

In [32]:
from sklearn.neural_network import MLPRegressor

In [33]:
model = on.Model(MLPRegressor(),
                 lags=30)
model.fit(ts)
sk_pred = model.predict(10)
sk_pred = sk_pred.rename({'pred': 'sklearn prediction'})

/home/benjy/projects_dev/ontime/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [34]:
# on a new single series (for now, inference can only be performed on single series)
model.predict(n=10, ts=new_ts)

<TimeSeries (DataArray) (time: 10, component: 1, sample: 1)> Size: 80B
array([[[-4.05516413]],

       [[-3.97963323]],

       [[-4.70101381]],

       [[-3.94403114]],

       [[-4.5825427 ]],

       [[-3.87290451]],

       [[-3.84491919]],

       [[-3.65995089]],

       [[-4.09419224]],

       [[-3.7710456 ]]])
Coordinates:
  * time       (time) datetime64[ns] 80B 2023-01-01 2023-01-02 ... 2023-01-10
  * component  (component) object 8B 'pred'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

## Using a PyTorch model

You can create and use custom PyTorch models and wrap them in our wrapper. It allows to quickly test its PyTorch model.  
For more complex behaviors, you can also create a new Darts model by inheriting one of the existing class in the library. You can find existing model implementations [here](https://github.com/unit8co/darts/tree/master/darts/models/forecasting).

In [35]:
import torch.nn as nn

In [36]:
class SimpleGRU(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, output_dim=1, num_layers=1):
        super(SimpleGRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_size = output_dim
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)  # Map hidden state to output dimension

    def forward(self, x):
        # Pass through GRU
        out, _ = self.gru(x)
        out = self.fc(out)   
        out = out[:, -6:, :] # the model output 6 features
        return out

In [37]:
model = on.Model(
    SimpleGRU,
    input_chunk_length=12,
    output_chunk_length=6, # should be equal what the output size the model was trained with
    num_layers=2,
    n_epochs=50,
        train_data_module_params={
        "val_split":0.2
    }
)

In [38]:
model.fit(ts)

INFO: GPU available: True (cuda), used: True
lightning.pytorch.utilities.rank_zero INFO  GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
lightning.pytorch.utilities.rank_zero INFO  TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
lightning.pytorch.utilities.rank_zero INFO  HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
lightning.pytorch.accelerators.cuda INFO  LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type      | Params | Mode 
----------------------------------------------
0 | model   | SimpleGRU | 9.7 K  | train
1 | loss_fn | MSELoss   | 0      | eval 
----------------------------------------------
9.7 K     Trainable params
0         Non-trainable params
9.7 K     Total params
0.039     Total estimated model params size (MB)
3         Modules in train mode
1         Modules in eval mode
lightning.pytorch.callbacks.model_summary INFO  
  | Name    | Type  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/benjy/projects_dev/ontime/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
lightning.pytorch.utilities.rank_zero INFO  `Trainer.fit` stopped: `max_epochs=50` reached.


In [39]:
torch_pred = model.predict(10)
torch_pred = torch_pred.rename({'0': 'torch prediction'})

/home/benjy/projects_dev/ontime/src/ontime/core/modelling/libs/pytorch/pytorch_forecasting_model.py:83: UserWarning: The requested prediction horizon (n=10) exceeds the model's output_chunk_length (6). The model will use an iterative forecasting approach, which may result in reduced accuracy due to error propagation.
  warnings.warn(


In [40]:
# on a new single series
model.predict(n=10, ts=new_ts)
# on many series
model.predict(n=10, ts=[new_ts]*3)

/home/benjy/projects_dev/ontime/src/ontime/core/modelling/libs/pytorch/pytorch_forecasting_model.py:83: UserWarning: The requested prediction horizon (n=10) exceeds the model's output_chunk_length (6). The model will use an iterative forecasting approach, which may result in reduced accuracy due to error propagation.
  warnings.warn(
/home/benjy/projects_dev/ontime/src/ontime/core/modelling/libs/pytorch/pytorch_forecasting_model.py:83: UserWarning: The requested prediction horizon (n=10) exceeds the model's output_chunk_length (6). The model will use an iterative forecasting approach, which may result in reduced accuracy due to error propagation.
  warnings.warn(


[<TimeSeries (DataArray) (time: 10, component: 1, sample: 1)> Size: 40B
 array([[[-4.725443 ]],
 
        [[-4.798504 ]],
 
        [[-5.324972 ]],
 
        [[-4.3734274]],
 
        [[-4.2623067]],
 
        [[-2.2508237]],
 
        [[-4.0632076]],
 
        [[-4.556101 ]],
 
        [[-4.762821 ]],
 
        [[-4.161046 ]]], dtype=float32)
 Coordinates:
   * time       (time) datetime64[ns] 80B 2023-01-01 2023-01-02 ... 2023-01-10
   * component  (component) <U1 4B '0'
 Dimensions without coordinates: sample
 Attributes:
     static_covariates:  None
     hierarchy:          None,
 <TimeSeries (DataArray) (time: 10, component: 1, sample: 1)> Size: 40B
 array([[[-4.725443 ]],
 
        [[-4.798504 ]],
 
        [[-5.324972 ]],
 
        [[-4.3734274]],
 
        [[-4.2623067]],
 
        [[-2.2508237]],
 
        [[-4.0632076]],
 
        [[-4.556101 ]],
 
        [[-4.762821 ]],
 
        [[-4.161046 ]]], dtype=float32)
 Coordinates:
   * time       (time) datetime64[ns] 80B 2023-0

## Comparing predictions

In [41]:
(on.Plot()
    .add(on.marks.line, ts[-50:])
    .add(on.marks.line, darts_pred)
    .add(on.marks.line, sk_pred)
    .add(on.marks.line, torch_pred)
    .properties(width=600, height=300)
    .show()
)

alt.LayerChart(...)